In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

df_data_1_C_plot = pd.read_csv("data/df_data_1_C_plot.csv")
df_data_1_C_plot.columns

df_data_1_C_plot = df_data_1_C_plot[df_data_1_C_plot['转炉终点Mn'] != 0]
print("when use 转炉终点Mn != 0 :", len(df_data_1_C_plot))
# df_data_1_C_plot.columns


df_data_1_C_plot_future = df_data_1_C_plot[['转炉终点温度', '转炉终点C', '转炉终点Mn', '转炉终点S', '转炉终点P', '转炉终点Si',
                                            '钢水净重', '氮化钒铁FeV55N11-A', '低铝硅铁', '钒氮合金(进口)',
                                            '钒铁(FeV50-B)', '硅铝钙', '硅铝合金FeAl30Si25',
                                            '硅锰面（硅锰渣）', '硅铁(合格块)', '硅铁FeSi75-B', '石油焦增碳剂',
                                            '锰硅合金FeMn64Si27(合格块)', '锰硅合金FeMn68Si18(合格块)', '碳化硅(55%)', '硅钙碳脱氧剂']]

from sklearn.preprocessing import StandardScaler
standardScaler = StandardScaler()
standardScaler.fit(df_data_1_C_plot_future)
df_data_1_C_plot_future_standardization = pd.DataFrame(standardScaler.transform(df_data_1_C_plot_future))
df_data_1_C_plot_future_standardization.columns = df_data_1_C_plot_future.columns

df_data_1_C_plot_label = df_data_1_C_plot['C_Yield']

when use 转炉终点Mn != 0 : 211


In [2]:
df_data_1_Mn_plot = pd.read_csv("data/df_data_1_Mn_plot.csv")
# df_data_1_Mn_plot.columns

df_data_1_Mn_plot_future = df_data_1_Mn_plot[['转炉终点温度', '转炉终点C', '转炉终点Mn', '转炉终点S', '转炉终点P', '转炉终点Si',
                                              '钢水净重', '氮化钒铁FeV55N11-A', '低铝硅铁', '钒氮合金(进口)',
                                              '钒铁(FeV50-B)', '硅铝钙', '硅铝合金FeAl30Si25',
                                              '硅锰面（硅锰渣）', '硅铁(合格块)', '硅铁FeSi75-B', '石油焦增碳剂',
                                              '锰硅合金FeMn64Si27(合格块)', '锰硅合金FeMn68Si18(合格块)', '碳化硅(55%)', '硅钙碳脱氧剂']]

standardScaler = StandardScaler()
standardScaler.fit(df_data_1_Mn_plot_future)
df_data_1_Mn_plot_future_standardization = pd.DataFrame(standardScaler.transform(df_data_1_Mn_plot_future))
df_data_1_Mn_plot_future_standardization.columns = df_data_1_Mn_plot_future.columns

df_data_1_Mn_plot_label = df_data_1_Mn_plot['Mn_Yield']

In [3]:
from sklearn import metrics
metrics.SCORERS.keys()

dict_keys(['adjusted_mutual_info_score', 'f1_weighted', 'f1', 'f1_macro', 'average_precision', 'neg_log_loss', 'explained_variance', 'balanced_accuracy', 'fowlkes_mallows_score', 'neg_mean_absolute_error', 'precision', 'recall_weighted', 'precision_weighted', 'recall_macro', 'accuracy', 'homogeneity_score', 'recall_micro', 'brier_score_loss', 'neg_mean_squared_log_error', 'v_measure_score', 'neg_mean_squared_error', 'normalized_mutual_info_score', 'mutual_info_score', 'precision_micro', 'r2', 'recall', 'f1_samples', 'precision_samples', 'neg_median_absolute_error', 'roc_auc', 'completeness_score', 'recall_samples', 'f1_micro', 'precision_macro', 'adjusted_rand_score'])

In [4]:
def Train_model(model,df_future=df_data_1_Mn_plot_future, df_label=df_data_1_Mn_plot_label,  y="Mn_Yield", scoring="r2"):

    # 训练模型，估计模型参数
    model.fit(df_future, df_label)

    from sklearn.model_selection import cross_val_score
    scores_clf_svc_cv = cross_val_score(model, df_future, df_label, cv=5, scoring=scoring)

    print(np.mean(scores_clf_svc_cv), scores_clf_svc_cv)

    return model

## 对Mn进行回归分析

In [5]:
from sklearn import linear_model
linearRegression = linear_model.LinearRegression()

linearRegression = Train_model(
    model=linearRegression,df_future=df_data_1_Mn_plot_future_standardization, scoring='neg_mean_squared_error')
linearRegression.coef_, linearRegression.intercept_

-0.0016005219704947715 [-0.00085179 -0.0002483  -0.00078778 -0.00110616 -0.00500858]


(array([-1.24416958e-03, -2.05757139e-04, -1.55451983e-02, -4.07091444e-03,
         3.83747713e-03, -1.42592942e-04,  3.15107730e-02,  6.49430222e-04,
         9.71445147e-17, -3.41295400e-03, -1.06332434e-02, -3.19721759e-03,
        -2.43977640e-03, -4.61035370e-02, -3.46944695e-17,  1.50122307e-02,
        -8.43659343e-04, -1.72720250e-01, -1.73745229e-01, -1.79490813e-03,
        -2.90007632e-03]), 0.8927604019550917)

In [6]:
np.sqrt(0.0067532713665304665 ),np.sqrt(0.0008860968349406865 )

(0.08217829011685791, 0.029767378704559904)

In [7]:
linearRegression  = Train_model(model=linearRegression,df_future=df_data_1_Mn_plot_future_standardization,scoring='r2')
linearRegression.coef_, linearRegression.intercept_
## 207 - 204 样本,不添加硅钙碳脱氧剂,而添加硅铁FeSi75-B, 导致其化学反应与其他样本差生了巨大差距

-1.041425498740198 [ 0.77363056  0.78571568  0.50091011 -0.47785734 -6.78952651]


(array([-1.24416958e-03, -2.05757139e-04, -1.55451983e-02, -4.07091444e-03,
         3.83747713e-03, -1.42592942e-04,  3.15107730e-02,  6.49430222e-04,
         9.71445147e-17, -3.41295400e-03, -1.06332434e-02, -3.19721759e-03,
        -2.43977640e-03, -4.61035370e-02, -3.46944695e-17,  1.50122307e-02,
        -8.43659343e-04, -1.72720250e-01, -1.73745229e-01, -1.79490813e-03,
        -2.90007632e-03]), 0.8927604019550917)

## 对C进行回归分析

In [8]:
from sklearn import linear_model
linearRegression = linear_model.LinearRegression()

linearRegression = Train_model(model=linearRegression, df_future=df_data_1_C_plot_future_standardization,
                               df_label=df_data_1_C_plot_label, scoring='neg_mean_squared_error')
linearRegression.coef_, linearRegression.intercept_

-0.0064147316023914856 [-0.00248512 -0.00306043 -0.00341139 -0.01990914 -0.00320758]


(array([-6.17937979e-04, -9.14410884e-02, -1.77883471e-03, -4.63004004e-03,
        -6.23187697e-03, -6.50159012e-03,  3.95624267e-02,  2.73872586e-03,
         7.11236625e-17,  3.88953501e-02,  1.96582567e-02, -1.24090108e-02,
        -2.63413169e-02, -1.08291855e-02, -1.04083409e-17,  7.78508783e-03,
        -6.76518346e-02, -3.66350193e-02, -4.42156497e-02, -1.57858584e-02,
        -5.94544035e-03]), 0.9026621151895744)

In [9]:
linearRegression = Train_model(model=linearRegression, df_future=df_data_1_C_plot_future_standardization,
                               df_label=df_data_1_C_plot_label, scoring='r2')
linearRegression.coef_, linearRegression.intercept_

-0.19499378652270297 [ 0.68390295  0.58897905  0.35861875 -2.93458994  0.32812027]


(array([-6.17937979e-04, -9.14410884e-02, -1.77883471e-03, -4.63004004e-03,
        -6.23187697e-03, -6.50159012e-03,  3.95624267e-02,  2.73872586e-03,
         7.11236625e-17,  3.88953501e-02,  1.96582567e-02, -1.24090108e-02,
        -2.63413169e-02, -1.08291855e-02, -1.04083409e-17,  7.78508783e-03,
        -6.76518346e-02, -3.66350193e-02, -4.42156497e-02, -1.57858584e-02,
        -5.94544035e-03]), 0.9026621151895744)

## 留一法测试
### Mn

In [11]:
from sklearn.model_selection import LeaveOneOut,LeavePOut
from sklearn.metrics import mean_squared_error,median_absolute_error,r2_score


linearRegression = linear_model.LinearRegression()

np_data_1_Mn_plot_future = np.array(df_data_1_Mn_plot_future)
np_data_1_Mn_plot_label = np.array(df_data_1_Mn_plot_label)

loo = LeavePOut(p=2)
loo.get_n_splits(np_data_1_Mn_plot_future)

mean_accuracy_score_list = []
mean_r2_score_list = []
for train_index, test_index in loo.split(np_data_1_Mn_plot_future):

    linearRegression.fit(
        np_data_1_Mn_plot_future[train_index], np_data_1_Mn_plot_label[train_index])

    prediction = linearRegression.predict(np_data_1_Mn_plot_future[test_index])

    mean_accuracy_score_list.append(mean_squared_error(
        np_data_1_Mn_plot_label[test_index], prediction))
    
#     linearRegression.
    mean_r2_score_list.append(r2_score(np_data_1_Mn_plot_label[test_index], prediction))
#     print(score)


print(np.average(mean_accuracy_score_list))
print(np.average(mean_r2_score_list))

0.0005586872599830876
-22940.99165554866


### 对C进行留一法测试

In [12]:
linearRegression = linear_model.LinearRegression()

np_data_1_C_plot_future = np.array(df_data_1_C_plot_future)
np_data_1_C_plot_label = np.array(df_data_1_C_plot_label)

loo = LeaveOneOut()
loo.get_n_splits(np_data_1_C_plot_future)


mean_accuracy_score_list = []
for train_index, test_index in loo.split(df_data_1_C_plot_future):

    linearRegression.fit(
        np_data_1_C_plot_future[train_index], np_data_1_C_plot_label[train_index])

    prediction = linearRegression.predict(np_data_1_C_plot_future[test_index])

    mean_accuracy_score_list.append(mean_squared_error(
        np_data_1_C_plot_label[test_index], prediction))


print(np.average(mean_accuracy_score_list))

0.002573623557835569


## 分析主要因素
### Mn

In [13]:
linearRegression = linear_model.LinearRegression()
linearRegression.fit(np_data_1_Mn_plot_future,np_data_1_Mn_plot_label)

predict = linearRegression.predict(np_data_1_Mn_plot_future)
print("MSE of predict ：", (mean_squared_error(y_true=np_data_1_Mn_plot_label,y_pred=predict)))

Mn_Predict = pd.DataFrame()
Mn_Predict['炉号'] = df_data_1_Mn_plot['炉号']
Mn_Predict['Mn_Yield_true'] = np_data_1_Mn_plot_label
Mn_Predict['Mn_Yield_predict'] = predict

Mn_Predict.to_csv("data/Mn_Predict.csv",index=None)
linearRegression.coef_, linearRegression.intercept_

MSE of predict ： 0.0004585632396820044


(array([-3.86861518e-05, -1.03296030e+00, -6.89444698e+01, -7.10807042e+01,
         5.56842284e+01, -1.90063086e-01,  1.06369391e-05,  9.69815973e-04,
         1.16271437e-09, -1.72249693e-04, -6.95630010e-04, -1.53242732e-04,
        -1.14362150e-04, -4.30639067e-04,  0.00000000e+00,  3.85187197e-04,
        -5.46570085e-05, -4.20669777e-04, -4.13164673e-04, -4.24908143e-05,
        -5.18331481e-04]), 0.9254543664760937)

### C

In [14]:
['转炉终点温度', '转炉终点C', '转炉终点Mn', '转炉终点S', '转炉终点P', '转炉终点Si',
'钢水净重', '氮化钒铁FeV55N11-A', '低铝硅铁', '钒氮合金(进口)',
'钒铁(FeV50-B)', '硅铝钙', '硅铝合金FeAl30Si25',
'硅锰面（硅锰渣）', '硅铁(合格块)', '硅铁FeSi75-B', '石油焦增碳剂',
'锰硅合金FeMn64Si27(合格块)', '锰硅合金FeMn68Si18(合格块)', '碳化硅(55%)', 
 '硅钙碳脱氧剂']

linearRegression = linear_model.LinearRegression()
linearRegression.fit(np_data_1_C_plot_future,np_data_1_C_plot_label)

predict = linearRegression.predict(np_data_1_C_plot_future)
print("MSE of predict ：", (mean_squared_error(y_true=np_data_1_C_plot_label,y_pred=predict)))

C_Predict = pd.DataFrame()
C_Predict['炉号'] = df_data_1_C_plot['炉号']
C_Predict['C_Yield_true'] = np_data_1_C_plot_label
C_Predict['C_Yield_predict'] = predict

C_Predict.to_csv("data/C_Predict.csv",index=None)
linearRegression.coef_, linearRegression.intercept_

MSE of predict ： 0.00208352869596435


(array([-1.87826420e-05, -4.63009258e+02, -7.94386856e+00, -8.09360295e+01,
        -9.03621035e+01, -8.63011550e+00,  1.33118073e-05,  3.98768983e-03,
         2.94884117e-09,  1.96693670e-03,  1.26982092e-03, -5.92586702e-04,
        -1.19389307e-03, -1.04514701e-04, -8.52651283e-14,  2.00946172e-04,
        -4.41113394e-03, -9.62602087e-05, -1.10781645e-04, -3.71786665e-04,
        -1.09222260e-03]), 0.8511816876001551)

## CI

In [15]:
import statsmodels.api as sm
import matplotlib.pyplot as plt


def evaluateModel(res):
    """
    分析线性回归模型的统计性质
    """
    # 整体统计分析结果
    print(res.summary())

def trainModel(X, Y):
    """
    训练模型
    """
    model = sm.OLS(Y, X)
    res = model.fit()
    return res


def confidenceInterval(_X,Y):
    # 加入常量变量
    X = sm.add_constant(_X)
    res = trainModel(X, Y)
    evaluateModel(res)

print("***************************************************")
# 利用 statsmodels 中的回归分析 对模型参数进行假设验证
print("加入不相关的新变量，新变量的系数被错误估计为不等于0")
print("***************************************************")
confidenceInterval(df_data_1_Mn_plot_future,df_data_1_Mn_plot_label)

***************************************************
加入不相关的新变量，新变量的系数被错误估计为不等于0
***************************************************
                            OLS Regression Results                            
Dep. Variable:               Mn_Yield   R-squared:                       0.764
Model:                            OLS   Adj. R-squared:                  0.742
Method:                 Least Squares   F-statistic:                     34.51
Date:                Sun, 14 Apr 2019   Prob (F-statistic):           5.29e-53
Time:                        20:23:33   Log-Likelihood:                 538.30
No. Observations:                 222   AIC:                            -1037.
Df Residuals:                     202   BIC:                            -968.5
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025    

In [16]:
print("***************************************************")
# 利用 statsmodels 中的回归分析 对模型参数进行假设验证
print("加入不相关的新变量，新变量的系数被错误估计为不等于0")
print("***************************************************")
confidenceInterval(df_data_1_C_plot_future,df_data_1_C_plot_label)

***************************************************
加入不相关的新变量，新变量的系数被错误估计为不等于0
***************************************************
                            OLS Regression Results                            
Dep. Variable:                C_Yield   R-squared:                       0.665
Model:                            OLS   Adj. R-squared:                  0.632
Method:                 Least Squares   F-statistic:                     20.00
Date:                Sun, 14 Apr 2019   Prob (F-statistic):           1.07e-35
Time:                        20:23:33   Log-Likelihood:                 351.93
No. Observations:                 211   AIC:                            -663.9
Df Residuals:                     191   BIC:                            -596.8
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025    

## 第二问 构建高准确率模型
### 数据格式转换

In [17]:
import xgboost as xgb
dtrain_C_train_data = xgb.DMatrix(data=df_data_1_C_plot_future,label=df_data_1_C_plot_label,missing=-999.0)
dtrain_Mn_train_data = xgb.DMatrix(data=df_data_1_Mn_plot_future,label=df_data_1_Mn_plot_label,missing=-999.0)

### 利用hyperopt对XGBoost构建

In [18]:
print("原有误差:")
print("指标 | C | Mn")
print("MSE",0.0021725786814569287," | ",0.0004894649188395173)
print("RMSE",np.sqrt(0.0021725786814569287)," | ",np.sqrt(0.0004894649188395173))

print("MSE",0.001736149557326879," | ",0.002110281505055306)
print("RMSE",np.sqrt(0.001736149557326879)," | ",np.sqrt(0.002110281505055306))

原有误差:
指标 | C | Mn
MSE 0.0021725786814569287  |  0.0004894649188395173
RMSE 0.046610928777025336  |  0.022123854068392273
MSE 0.001736149557326879  |  0.002110281505055306
RMSE 0.04166712801870173  |  0.04593780039417763


### 对C的模型进行优化

In [41]:
import hyperopt
import xgboost as xgb

params_space = {
    'max_depth': hyperopt.hp.randint('max_depth', 12),
    'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 5e-1),
    'n_estimators':hyperopt.hp.randint('n_estimators', 300)
}

from numpy.random import RandomState

In [19]:
def hyperopt_objective(params):

    model = xgb.XGBRegressor(
        max_depth=int(params['max_depth']) + 1,
        learning_rate=params['learning_rate'],
        silent=1,
        objective='reg:linear',
        eval_metric='rmse',
        seed=42,
        nthread=-1,
    )

    res = xgb.cv(model.get_params(), dtrain_C_train_data, num_boost_round=params['n_estimators'], nfold=5,
                 callbacks=[xgb.callback.print_evaluation(show_stdv=False),
                            xgb.callback.early_stop(15)])

    return np.min(res['test-rmse-mean'])  # as hyperopt minimises

trials = hyperopt.Trials()

best = hyperopt.fmin(
    hyperopt_objective,
    space=params_space,
    algo=hyperopt.tpe.suggest,
    max_evals=50,
    trials=trials,
    rstate=RandomState(42)
)

print("\n展示hyperopt获取的最佳结果，但是要注意的是我们对hyperopt最初的取值范围做过一次转换")
print(best)

[0]	train-rmse:0.289073	test-rmse:0.288849                                                                             

Multiple eval metrics have been passed: 'test-rmse' will be used for early stopping.                                   


Will train until test-rmse hasn't improved in 15 rounds.                                                               

[1]	train-rmse:0.203057	test-rmse:0.204082                                                                             

[2]	train-rmse:0.145003	test-rmse:0.149959                                                                             

[3]	train-rmse:0.105476	test-rmse:0.117058                                                                             

[4]	train-rmse:0.0779744	test-rmse:0.0961894                                                                           

[5]	train-rmse:0.058352	test-rmse:0.0834796                                                                            

[6]	train-rmse:0.0439052	test-r

[54]	train-rmse:0.0011204	test-rmse:0.0631576                                                                          

[55]	train-rmse:0.0010448	test-rmse:0.063165                                                                           

[56]	train-rmse:0.0009844	test-rmse:0.063165                                                                           

[57]	train-rmse:0.0009304	test-rmse:0.0631668                                                                          

[58]	train-rmse:0.0008812	test-rmse:0.063168                                                                           

[59]	train-rmse:0.0008444	test-rmse:0.0631656                                                                          

[60]	train-rmse:0.000809	test-rmse:0.0631636                                                                           

[61]	train-rmse:0.0007734	test-rmse:0.0631556                                                                          

[62]	train-rmse:0.0007536	test-r

[31]	train-rmse:0.027319	test-rmse:0.062577                                                                            

[32]	train-rmse:0.0267098	test-rmse:0.062529                                                                           

[33]	train-rmse:0.0261784	test-rmse:0.0625744                                                                          

[34]	train-rmse:0.0257236	test-rmse:0.0626036                                                                          

[35]	train-rmse:0.025303	test-rmse:0.0624242                                                                           

[36]	train-rmse:0.0248238	test-rmse:0.0621666                                                                          

[37]	train-rmse:0.0243268	test-rmse:0.0621506                                                                          

[38]	train-rmse:0.0239866	test-rmse:0.0621072                                                                          

[39]	train-rmse:0.0235206	test-r

[53]	train-rmse:0.0006014	test-rmse:0.0630212                                                                          

[54]	train-rmse:0.0006014	test-rmse:0.0630212                                                                          

[55]	train-rmse:0.0006014	test-rmse:0.0630212                                                                          

[56]	train-rmse:0.0006014	test-rmse:0.0630212                                                                          

[57]	train-rmse:0.0006014	test-rmse:0.0630212                                                                          

[58]	train-rmse:0.0006014	test-rmse:0.0630212                                                                          

[59]	train-rmse:0.0006014	test-rmse:0.0630212                                                                          

[60]	train-rmse:0.0006014	test-rmse:0.0630212                                                                          

[61]	train-rmse:0.0006014	test-r

[9]	train-rmse:0.073615	test-rmse:0.0919322                                                                            

[10]	train-rmse:0.0631	test-rmse:0.084535                                                                              

[11]	train-rmse:0.0544384	test-rmse:0.0795778                                                                          

[12]	train-rmse:0.0470198	test-rmse:0.0758626                                                                          

[13]	train-rmse:0.0407772	test-rmse:0.0730108                                                                          

[14]	train-rmse:0.0355626	test-rmse:0.070657                                                                           

[15]	train-rmse:0.0309794	test-rmse:0.0690492                                                                          

[16]	train-rmse:0.0271196	test-rmse:0.0676782                                                                          

[17]	train-rmse:0.023813	test-rm

...

[16]	train-rmse:0.0545076	test-rmse:0.0656772                                                                          

[17]	train-rmse:0.0539608	test-rmse:0.0653436                                                                          

[18]	train-rmse:0.0534524	test-rmse:0.0654938                                                                          

[19]	train-rmse:0.0529658	test-rmse:0.0646166                                                                          

[20]	train-rmse:0.05256	test-rmse:0.0645994                                                                            

[21]	train-rmse:0.0521376	test-rmse:0.0644548                                                                          

[0]	train-rmse:0.231651	test-rmse:0.232321                                                                             

Multiple eval metrics have been passed: 'test-rmse' will be used for early stopping.                                   


Will train until test-rmse hasn

[19]	train-rmse:0.0006488	test-rmse:0.0653638                                                                          

[20]	train-rmse:0.0006222	test-rmse:0.065359                                                                           

[21]	train-rmse:0.0006192	test-rmse:0.0653594                                                                          

[22]	train-rmse:0.0006188	test-rmse:0.0653588                                                                          

[23]	train-rmse:0.0006188	test-rmse:0.0653588                                                                          

[24]	train-rmse:0.0006188	test-rmse:0.0653584                                                                          

[25]	train-rmse:0.0006188	test-rmse:0.0653584                                                                          

[26]	train-rmse:0.0006188	test-rmse:0.0653582                                                                          

[27]	train-rmse:0.0006188	test-r

[48]	train-rmse:0.0125486	test-rmse:0.0608792                                                                          

[49]	train-rmse:0.0122508	test-rmse:0.0609274                                                                          

[50]	train-rmse:0.0118968	test-rmse:0.0609158                                                                          

[51]	train-rmse:0.0115592	test-rmse:0.0609708                                                                          

[52]	train-rmse:0.0112772	test-rmse:0.060999                                                                           

[53]	train-rmse:0.0110762	test-rmse:0.061037                                                                           

[54]	train-rmse:0.0108672	test-rmse:0.061052                                                                           

[55]	train-rmse:0.0107242	test-rmse:0.061057                                                                           

[56]	train-rmse:0.0104762	test-r

[1]	train-rmse:0.162632	test-rmse:0.163843                                                                             

[2]	train-rmse:0.106478	test-rmse:0.1153                                                                               

[3]	train-rmse:0.07207	test-rmse:0.0892502                                                                             

[4]	train-rmse:0.0494406	test-rmse:0.0778058                                                                           

[5]	train-rmse:0.0344396	test-rmse:0.074033                                                                            

[6]	train-rmse:0.024588	test-rmse:0.0714714                                                                            

[7]	train-rmse:0.017657	test-rmse:0.0700812                                                                            

[8]	train-rmse:0.0127498	test-rmse:0.0691188                                                                           

[9]	train-rmse:0.0092512	test-rm

[61]	train-rmse:0.0005882	test-rmse:0.0641964                                                                          

[62]	train-rmse:0.0005882	test-rmse:0.0641964                                                                          

Stopping. Best iteration:                                                                                              
[47]	train-rmse:0.000592+2.9278e-05	test-rmse:0.0641958+0.00568424


100%|██████████████████████████████████████████████████| 50/50 [00:37<00:00,  1.52it/s, best loss: 0.05942839999999999]

展示hyperopt获取的最佳结果，但是要注意的是我们对hyperopt最初的取值范围做过一次转换
{'n_estimators': 115, 'learning_rate': 0.40909717981221194, 'max_depth': 2}


### 使用sklearn + Hyperopt 对XGBoost进行超参数优化

In [42]:
import hyperopt

def XGBRegressor_CV(params):
    from sklearn.model_selection import cross_val_score
    
    model = xgb.XGBRegressor(
        max_depth=int(params['max_depth']) + 1,
        learning_rate=params['learning_rate'],
        n_estimators=params['n_estimators'],
        silent=1,
        objective='reg:linear',
        eval_metric='rmse',
        seed=42,
        nthread=-1,
        early_stopping_rounds=15
    )

    metric = cross_val_score(model, df_data_1_C_plot_future,df_data_1_C_plot_label, cv=5, scoring="neg_mean_squared_error")

    return np.mean(-metric)

trials_2 = hyperopt.Trials()

best_2_C = hyperopt.fmin(
    XGBRegressor_CV,
    space=params_space,
    algo=hyperopt.tpe.suggest,
    max_evals=300,
    trials=trials_2,
    rstate=RandomState(42)
)

print("\n展示hyperopt获取的最佳结果，但是要注意的是我们对hyperopt最初的取值范围做过一次转换")
print(best_2_C)

100%|███████████████████████████████████████████████| 300/300 [02:26<00:00,  2.00it/s, best loss: 0.004336069555406805]

展示hyperopt获取的最佳结果，但是要注意的是我们对hyperopt最初的取值范围做过一次转换
{'max_depth': 0, 'n_estimators': 59, 'learning_rate': 0.1019316920558744}


### 得到新的模型

In [43]:
C_CV_Model = xgb.XGBRegressor(
    max_depth= best_2_C["max_depth"] + 1,
    learning_rate=best_2_C["learning_rate"],
    n_estimators= best_2_C['n_estimators'],
    silent=1,
    objective='reg:linear',
    eval_metric='rmse',
    seed=42,
    nthread=-1,
    early_stopping_rounds=15
)

C_CV_Model.fit( df_data_1_C_plot_future,df_data_1_C_plot_label)

C_CV_Model.save_model('model/C_CV_Model.model')

In [87]:
C_CV_Model_new = xgb.Booster({'nthread': -1})  # init model
C_CV_Model_new.load_model('model/C_CV_Model.model')  # load data

dpredict = xgb.DMatrix(df_data_1_C_plot_future)
predict = C_CV_Model_new.predict(dpredict)

C_Predict_XGB = pd.DataFrame()
C_Predict_XGB['炉号'] = df_data_1_C_plot['炉号']
C_Predict_XGB['C_Yield_true'] = np_data_1_C_plot_label
C_Predict_XGB['C_Yield_predict'] = predict

C_Predict_XGB.to_csv("data/C_Predict_XGB.csv",index=None)

from sklearn.metrics import mean_squared_error
print("MSE of predict ：", (mean_squared_error(y_true=df_data_1_C_plot_label,y_pred=predict)))

MSE of predict ： 0.0031119961709577797


### 对Mn的模型进行优化

In [23]:
import hyperopt


def hyperopt_objective(params):

    model = xgb.XGBRegressor(
        max_depth=int(params['max_depth']) + 1,
        learning_rate=params['learning_rate'],
        silent=1,
        objective='reg:linear',
        eval_metric='rmse',
        seed=42,
        nthread=-1,
    )

    res = xgb.cv(model.get_params(), dtrain_Mn_train_data, num_boost_round=params['n_estimators'], nfold=5,
                 callbacks=[xgb.callback.print_evaluation(show_stdv=False),
                            xgb.callback.early_stop(15)])

    return np.min(res['test-rmse-mean'])  # as hyperopt minimises

from numpy.random import RandomState

params_space = {
    'max_depth': hyperopt.hp.randint('max_depth', 12),
    'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 5e-1),
    'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 5e-1),
    'n_estimators':hyperopt.hp.randint('n_estimators', 300)
}

trials = hyperopt.Trials()

best = hyperopt.fmin(
    hyperopt_objective,
    space=params_space,
    algo=hyperopt.tpe.suggest,
    max_evals=50,
    trials=trials,
    rstate=RandomState(42)
)

print("\n展示hyperopt获取的最佳结果，但是要注意的是我们对hyperopt最初的取值范围做过一次转换")
print(best)

[0]	train-rmse:0.270962	test-rmse:0.271323                                                                             

Multiple eval metrics have been passed: 'test-rmse' will be used for early stopping.                                   


Will train until test-rmse hasn't improved in 15 rounds.                                                               

[1]	train-rmse:0.187082	test-rmse:0.187895                                                                             

[2]	train-rmse:0.130332	test-rmse:0.132273                                                                             

[3]	train-rmse:0.091895	test-rmse:0.0961534                                                                            

[4]	train-rmse:0.0656726	test-rmse:0.0731704                                                                           

[5]	train-rmse:0.0476094	test-rmse:0.059159                                                                            

[6]	train-rmse:0.0350992	test-r

[11]	train-rmse:0.005193	test-rmse:0.0422138                                                                           

[12]	train-rmse:0.004372	test-rmse:0.0423226                                                                           

[13]	train-rmse:0.0038356	test-rmse:0.042342                                                                           

[14]	train-rmse:0.00335	test-rmse:0.0423036                                                                            

[15]	train-rmse:0.0029388	test-rmse:0.0422962                                                                          

[16]	train-rmse:0.0026088	test-rmse:0.0423198                                                                          

[17]	train-rmse:0.0021178	test-rmse:0.0422582                                                                          

[18]	train-rmse:0.0018584	test-rmse:0.042285                                                                           

[19]	train-rmse:0.001608	test-rm

[52]	train-rmse:0.0096648	test-rmse:0.0370992                                                                          

[53]	train-rmse:0.0095432	test-rmse:0.037041                                                                           

[54]	train-rmse:0.0093906	test-rmse:0.0370126                                                                          

[55]	train-rmse:0.0092898	test-rmse:0.037037                                                                           

[56]	train-rmse:0.0091506	test-rmse:0.0370086                                                                          

[57]	train-rmse:0.0090574	test-rmse:0.0370198                                                                          

[58]	train-rmse:0.00893	test-rmse:0.0370232                                                                            

[59]	train-rmse:0.0088126	test-rmse:0.037025                                                                           

[60]	train-rmse:0.0086584	test-r

[21]	train-rmse:0.0031372	test-rmse:0.0391654                                                                          

[22]	train-rmse:0.0026674	test-rmse:0.0391106                                                                          

[23]	train-rmse:0.0022656	test-rmse:0.0391012                                                                          

[24]	train-rmse:0.0019476	test-rmse:0.0390606                                                                          

[25]	train-rmse:0.0016688	test-rmse:0.0390504                                                                          

[26]	train-rmse:0.0014406	test-rmse:0.0390348                                                                          

[27]	train-rmse:0.0012502	test-rmse:0.039017                                                                           

[28]	train-rmse:0.0010902	test-rmse:0.0390114                                                                          

[29]	train-rmse:0.0009588	test-r

[2]	train-rmse:0.0892018	test-rmse:0.0926966                                                                           

[3]	train-rmse:0.0578944	test-rmse:0.0655136                                                                           

[4]	train-rmse:0.0402534	test-rmse:0.0509824                                                                           

[5]	train-rmse:0.0308172	test-rmse:0.0444896                                                                           

[6]	train-rmse:0.0254888	test-rmse:0.0416332                                                                           

[7]	train-rmse:0.0225018	test-rmse:0.0403986                                                                           

[8]	train-rmse:0.020696	test-rmse:0.0396338                                                                            

[9]	train-rmse:0.0194752	test-rmse:0.0395462                                                                           

[10]	train-rmse:0.0182142	test-r

...

[38]	train-rmse:0.000723	test-rmse:0.0422832                                                                           

[39]	train-rmse:0.000723	test-rmse:0.0422832                                                                           

Stopping. Best iteration:                                                                                              
[24]	train-rmse:0.000723+4.38224e-05	test-rmse:0.042283+0.0174387


[0]	train-rmse:0.214883	test-rmse:0.215637                                                                             

Multiple eval metrics have been passed: 'test-rmse' will be used for early stopping.                                   


Will train until test-rmse hasn't improved in 15 rounds.                                                               

[1]	train-rmse:0.119507	test-rmse:0.121935                                                                             

[2]	train-rmse:0.069027	test-rmse:0.076229                                           

[41]	train-rmse:0.0246276	test-rmse:0.0382948                                                                          

[42]	train-rmse:0.0244838	test-rmse:0.0382978                                                                          

[43]	train-rmse:0.0243388	test-rmse:0.038322                                                                           

[44]	train-rmse:0.0242046	test-rmse:0.038251                                                                           

[45]	train-rmse:0.02407	test-rmse:0.0381654                                                                            

[46]	train-rmse:0.0239366	test-rmse:0.0380108                                                                          

[47]	train-rmse:0.0238044	test-rmse:0.0379446                                                                          

[48]	train-rmse:0.0236812	test-rmse:0.0378998                                                                          

[49]	train-rmse:0.0235594	test-r

[175]	train-rmse:0.0167014	test-rmse:0.0355968                                                                         

[176]	train-rmse:0.0166738	test-rmse:0.035574                                                                          

[177]	train-rmse:0.0166456	test-rmse:0.035598                                                                          

[178]	train-rmse:0.01662	test-rmse:0.0355568                                                                           

[179]	train-rmse:0.016592	test-rmse:0.0355336                                                                          

[180]	train-rmse:0.016564	test-rmse:0.035535                                                                           

[181]	train-rmse:0.016538	test-rmse:0.0356152                                                                          

[182]	train-rmse:0.016511	test-rmse:0.03554                                                                            

[183]	train-rmse:0.0164856	test-

[8]	train-rmse:0.0143072	test-rmse:0.0421674                                                                           

[9]	train-rmse:0.0112982	test-rmse:0.0416236                                                                           

[10]	train-rmse:0.0089448	test-rmse:0.0410898                                                                          

[11]	train-rmse:0.0073668	test-rmse:0.0413202                                                                          

[12]	train-rmse:0.0061672	test-rmse:0.0415052                                                                          

[13]	train-rmse:0.0050954	test-rmse:0.0416132                                                                          

[14]	train-rmse:0.0044526	test-rmse:0.0416778                                                                          

[15]	train-rmse:0.0039104	test-rmse:0.0416498                                                                          

[16]	train-rmse:0.0033208	test-r


[0]	train-rmse:0.21166	test-rmse:0.212444                                                                              

Multiple eval metrics have been passed: 'test-rmse' will be used for early stopping.                                   


Will train until test-rmse hasn't improved in 15 rounds.                                                               

[1]	train-rmse:0.116154	test-rmse:0.118702                                                                             

[2]	train-rmse:0.0664394	test-rmse:0.0739278                                                                           

[3]	train-rmse:0.0400216	test-rmse:0.0545704                                                                           

[4]	train-rmse:0.0254718	test-rmse:0.0466188                                                                           

[5]	train-rmse:0.0169852	test-rmse:0.043958                                                                            

[6]	train-rmse:0.011796	test-r

### 使用sklearn + Hyperopt 对XGBoost进行超参数优化

In [24]:
def XGBRegressor_CV(params):
    from sklearn.model_selection import cross_val_score
    
    model = xgb.XGBRegressor(
        max_depth=int(params['max_depth']) + 1,
        learning_rate=params['learning_rate'],
        n_estimators=params['n_estimators'],
        silent=1,
        objective='reg:linear',
        eval_metric='rmse',
        seed=42,
        nthread=-1,
        early_stopping_rounds=15
    )

    metric = cross_val_score(model, df_data_1_Mn_plot_future,df_data_1_Mn_plot_label, cv=5, scoring="neg_mean_squared_error")

    return np.mean(-metric)

trials_2 = hyperopt.Trials()

best_2_Mn = hyperopt.fmin(
    XGBRegressor_CV,
    space=params_space,
    algo=hyperopt.tpe.suggest,
    max_evals=300,
    trials=trials_2,
    rstate=RandomState(42)
)

print("\n展示hyperopt获取的最佳结果，但是要注意的是我们对hyperopt最初的取值范围做过一次转换")
print(best_2_Mn)

100%|███████████████████████████████████████████████| 300/300 [02:29<00:00,  1.55it/s, best loss: 0.002110281505055306]

展示hyperopt获取的最佳结果，但是要注意的是我们对hyperopt最初的取值范围做过一次转换
{'n_estimators': 293, 'learning_rate': 0.4996319872715388, 'max_depth': 0}


### 得到新的模型

In [25]:
Mn_CV_Model = xgb.XGBRegressor(
    max_depth= best_2_Mn["max_depth"] + 1,
    learning_rate=best_2_Mn["learning_rate"],
    n_estimators= best_2_Mn['n_estimators'],
    silent=1,
    objective='reg:linear',
    eval_metric='rmse',
    seed=42,
    nthread=-1,
    early_stopping_rounds=15
)

Mn_CV_Model.fit( df_data_1_Mn_plot_future,df_data_1_Mn_plot_label)

Mn_CV_Model.save_model('model/Mn_CV_Model.model')

In [88]:
Mn_CV_Model_new = xgb.Booster({'nthread': -1})  # init model
Mn_CV_Model_new.load_model('model/Mn_CV_Model.model')  # load data

dpredict = xgb.DMatrix(df_data_1_Mn_plot_future)
predict = Mn_CV_Model_new.predict(dpredict)

Mn_Predict_XGB = pd.DataFrame()
Mn_Predict_XGB['炉号'] = df_data_1_Mn_plot['炉号']
Mn_Predict_XGB['C_Yield_true'] = np_data_1_Mn_plot_label
Mn_Predict_XGB['C_Yield_predict'] = predict

Mn_Predict_XGB.to_csv("data/Mn_Predict_XGB.csv",index=None)

from sklearn.metrics import mean_squared_error
print("MSE of predict ：", (mean_squared_error(y_true=df_data_1_Mn_plot_label,y_pred=predict)))

MSE of predict ： 0.00019115669935288612
